# Get Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime
import requests
import json


CLIENT_ID='23QQNX'
CLIENT_SECRET='2cf8156ff7b81eadd26d80a30082067c'

# API Authorization

In [2]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[12/Apr/2023:16:30:14] ENGINE Listening for SIGTERM.
[12/Apr/2023:16:30:14] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[12/Apr/2023:16:30:14] ENGINE Set handler for console events.
[12/Apr/2023:16:30:14] ENGINE Started monitor thread 'Autoreloader'.
[12/Apr/2023:16:30:15] ENGINE Serving on http://127.0.0.1:8080
[12/Apr/2023:16:30:15] ENGINE Bus STARTED


127.0.0.1 - - [12/Apr/2023:16:30:17] "GET /?code=3f6a07571fcb5022f6da671d3f170bc2df8077aa&state=eEePKwCQLRaivWihRviCw92ng9ojDX HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"


[12/Apr/2023:16:30:18] ENGINE Bus STOPPING
[12/Apr/2023:16:30:18] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[12/Apr/2023:16:30:18] ENGINE Removed handler for console events.
[12/Apr/2023:16:30:18] ENGINE Stopped thread 'Autoreloader'.
[12/Apr/2023:16:30:18] ENGINE Bus STOPPED
[12/Apr/2023:16:30:18] ENGINE Bus EXITING
[12/Apr/2023:16:30:18] ENGINE Waiting for child threads to terminate...
[12/Apr/2023:16:30:18] ENGINE Bus EXITED
[12/Apr/2023:16:30:18] ENGINE Waiting for thread Thread-18.


# Get Data

### Automated process with a function to access each of the variables

In [3]:
def df_fitbit(activity, base_date, end_date, token):
    url = 'https://api.fitbit.com/1/user/-/' + activity + '/date/' + base_date + '/' + end_date + '.json'
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()

    return response

In [4]:
token = ACCESS_TOKEN
base_date = '2023-03-29'
end_date = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

#data to gather
activity = ['activities/minutesSedentary', 'activities/log/minutesLightlyActive', 'activities/log/minutesFairlyActive', 
      'activities/log/minutesVeryActive']

sleep = ['sleep/startTime', 'sleep/timeInBed', 'sleep/minutesAsleep', 'sleep/minutesAwake', 'sleep/efficiency']

### Get User Engagement Data

In [5]:
minutesSedentary = df_fitbit(activity[0], base_date, end_date, token)
#print(minutesSedentary)
minutesLightlyActive = df_fitbit(activity[1], base_date, end_date, token)
#print(minutesLightlyActive)
minutesFairlyActive = df_fitbit(activity[2], base_date, end_date, token)
#print(minutesFairlyActive)
minutesVeryActive = df_fitbit(activity[3], base_date, end_date, token)
#print(minutesVeryActive)

### Get Sleep Data

In [6]:
StartTime = df_fitbit(sleep[0], base_date, end_date, token)
#print(StartTime)
timeInBed = df_fitbit(sleep[1], base_date, end_date, token)
#print(timeInBed)
minutesAsleep = df_fitbit(sleep[2], base_date, end_date, token)
#print(minutesAsleep)
minutesAwake = df_fitbit(sleep[3], base_date, end_date, token)
#print(minutesAwake)
efficiency = df_fitbit(sleep[4], base_date, end_date, token)
#print(efficiency)

### Transform Data into the appropriate form

In [7]:
import hashlib
import random

def create_data(dicts,types):
    data_list = []
    # loop through each nested dictionary
    for i in range(0,len(dicts)):        
        for item in dicts[i]:            
            # create a random hash ID
            hash_object = hashlib.sha256(str(random.getrandbits(256)).encode())
            id = hash_object.hexdigest()           
            # create a new dictionary with the id, type, and data fields
            new_dict = {
                'id': id,
                'type': types[i],
                'data': item
            }           
            # append the new dictionary to the output list
            data_list.append(new_dict)
    return data_list     

In [8]:
#gather all activity dictionaries in a list
act_dicts = [minutesSedentary['activities-minutesSedentary'], minutesLightlyActive['activities-log-minutesLightlyActive'], 
              minutesFairlyActive['activities-log-minutesFairlyActive'], minutesVeryActive['activities-log-minutesVeryActive']]
act_types = ['minutesSedentary', 'minutesLightlyActive', 'minutesFairlyActive', 'minutesVeryActive']

activity_data = create_data(act_dicts,act_types)
activity_data

[{'id': '59ac20c01df84e7e50e5bf9e42ea8ae5bf72b8d57bb00b73943611afbdfc6144',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-03-29', 'value': '727'}},
 {'id': 'c19692b0a320d83443bce0fb52d7768b444ac0d9103e18457d8d408e03f2cdaa',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-03-30', 'value': '776'}},
 {'id': '4cf7f561af0f2681e8767b1d19f402b52328ec5579a0ae0e046a9459894d3b62',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-03-31', 'value': '730'}},
 {'id': '606346a480051599f529f5f176154562d3432664a3fe24bcb860210b3473fa74',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-04-01', 'value': '753'}},
 {'id': '04933cf25a3061ce67a030f14f7205fc3e2eeaf1ab60d6ce158ea3aaf46c256c',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-04-02', 'value': '745'}},
 {'id': 'c282b144329db5f220615cfc06c0fe887aebf1e0f29bd5d967656d3aa4fbb906',
  'type': 'minutesSedentary',
  'data': {'dateTime': '2023-04-03', 'value': '823'}},
 {'id': '666a1e8cfbf2f62961df23f7a

In [9]:
#gather all sleep dictionaries in a list
sleep_dicts = [StartTime['sleep-startTime'], timeInBed['sleep-timeInBed'], minutesAsleep['sleep-minutesAsleep'], 
             minutesAwake['sleep-minutesAwake'], efficiency['sleep-efficiency']]
sleep_types = ['startTime', 'timeInBed', 'minutesAsleep', 'minutesAwake','efficiency']

sleep_data = create_data(sleep_dicts,sleep_types)
sleep_data

[{'id': '99b716d89e5769073991acca89573243081d6db98540299862a4e16ad74e8b22',
  'type': 'startTime',
  'data': {'dateTime': '2023-03-29', 'value': '00:34'}},
 {'id': '5bad8b01393ec577712f157eda01b67937cbcb2a322a94c32d86937de5f77c43',
  'type': 'startTime',
  'data': {'dateTime': '2023-03-30', 'value': '00:52'}},
 {'id': '00e59fd5f530bbaeef8e262e5dab0774fd66dc430c11f9b1831fb29d6b2594d7',
  'type': 'startTime',
  'data': {'dateTime': '2023-03-31', 'value': '00:23'}},
 {'id': '32dff587b3d6d3d509c13f3d1c7651149261c823ee33ab419b4ee22c6acca79c',
  'type': 'startTime',
  'data': {'dateTime': '2023-04-01', 'value': '03:01'}},
 {'id': '9076acdd9c7b09d9718ceefd02f253a328bf61cca4417b940186abea14d6fc3e',
  'type': 'startTime',
  'data': {'dateTime': '2023-04-02', 'value': '05:08'}},
 {'id': '96a689c8aa952c19f31ba1b2f91d65c5603a21943525e400a9a8f1d71408d48b',
  'type': 'startTime',
  'data': {'dateTime': '2023-04-03', 'value': '23:28'}},
 {'id': 'afaa0fb16d28330ea5afa1521be6bb688d488ee212e71a9de4b7366

In [10]:
# do not pay attention. just trying some things
'''
def df_fitbit_2(token):
    url = 'https://api.fitbit.com/1/user/-/activities/goals/weekly.json'    
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()
    return response

def df_fitbit_3(base_date, token):
    url = 'https://api.fitbit.com/1/user/-/activities/date/' + base_date + '.json'
    response = requests.get(url=url, headers={'Authorization':'Bearer ' + token}).json()
    return response

df_fitbit_2(token)

result = df_fitbit_3(base_date, token)
result['goals']

allDates = pd.date_range(start=base_date, end = end_date)

for oneDate in allDates:
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    oneDay = df_fitbit_3(oneDate, token)
    oneDayGoals = oneDay['goals']
    print(oneDate)
    print(oneDayGoals)
'''

'\ndef df_fitbit_2(token):\n    url = \'https://api.fitbit.com/1/user/-/activities/goals/weekly.json\'    \n    response = requests.get(url=url, headers={\'Authorization\':\'Bearer \' + token}).json()\n    return response\n\ndef df_fitbit_3(base_date, token):\n    url = \'https://api.fitbit.com/1/user/-/activities/date/\' + base_date + \'.json\'\n    response = requests.get(url=url, headers={\'Authorization\':\'Bearer \' + token}).json()\n    return response\n\ndf_fitbit_2(token)\n\nresult = df_fitbit_3(base_date, token)\nresult[\'goals\']\n\nallDates = pd.date_range(start=base_date, end = end_date)\n\nfor oneDate in allDates:\n    oneDate = oneDate.date().strftime("%Y-%m-%d")\n    oneDay = df_fitbit_3(oneDate, token)\n    oneDayGoals = oneDay[\'goals\']\n    print(oneDate)\n    print(oneDayGoals)\n'

# Inserting data into MongoDB 

In [10]:
# importing pymongo
from pymongo import MongoClient

# establing connection
try:
    connect = MongoClient()
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")

# connecting or switching to the database
db = connect.fitbitDB

# creating or switching to demoCollection
collection = db.demoCollection

Connected successfully!!!


In [12]:
# Inserting activity documents one by one
for document in activity_data:
    collection.insert_one(document)

In [13]:
# Inserting sleep documents one by one
for document in sleep_data:
    collection.insert_one(document)

# Data Visualization with Streamlit

In [7]:
#!pip install streamlit

In [9]:
import streamlit as st

In [11]:
data = collection.find()  

# Loop through the retrieved data and display it in Streamlit
for document in data:
    # Access document fields and display the data in Streamlit components
    # (e.g., st.write(), st.table(), etc.)
    st.write(document)

2023-04-18 16:38:16.068 
  command:

    streamlit run C:\Users\30698\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [12]:
import asyncio
import pymongo
import streamlit as st

# Create and set an event loop
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)

# Connect to MongoDB
client = pymongo.MongoClient()
db = client.fitbitDB

# Retrieve data and display in Streamlit
collection = db.demoCollection
data = collection.find()
for document in data:
    st.write(document)